In [22]:
%load_ext autoreload
%autoreload 2

import geopandas as gpd
import matplotlib.pyplot as plt 
import jax.numpy as jnp
import numpy as onp
import pyreadr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
file = pyreadr.read_r("data/PRISM/ny_tmean/weighted_area_raster_zip_36_tmean_daily_1995.rds")


df = gpd.GeoDataFrame(file[None])


In [47]:
df["zcta"] = df["zcta"].astype("string")

In [24]:
onp.unique(df["zcta"])

array(['06390', '10001', '10002', ..., '14903', '14904', '14905'],
      dtype=object)

In [25]:
654810 /1794

365.0

In [26]:
zipfile = "zip://data/tl_2010_us_zcta510.zip"
all_zctas = gpd.read_file(zipfile)

In [27]:
all_zctas["ZCTA5CE10"]

0        04757
1        04758
2        04760
3        04975
4        04976
         ...  
33115    05456
33116    05457
33117    05458
33118    05459
33119    05461
Name: ZCTA5CE10, Length: 33120, dtype: object

In [34]:
all_zctas_centroids = gpd.GeoDataFrame(data = all_zctas["ZCTA5CE10"], geometry = all_zctas.geometry.to_crs("EPSG:4269").centroid)

all_zctas_centroids.to_file("data/zcta_centroids.shp")

/tmp/ipykernel_28179/3709664977.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  all_zctas_centroids = gpd.GeoDataFrame(data = all_zctas["ZCTA5CE10"], geometry = all_zctas.geometry.to_crs("EPSG:4269").centroid)


In [35]:
all_zctas
all_zctas_centroids

,ZCTA5CE10,geometry
0,04757,POINT (-68.17241 46.66262)
1,04758,POINT (-67.84723 46.55424)
2,04760,POINT (-67.93639 46.34674)
3,04975,POINT (-69.58833 44.62535)
4,04976,POINT (-69.66953 44.79877)
...,...,...
33115,05456,POINT (-73.25508 44.20763)
33116,05457,POINT (-72.91338 44.97100)
33117,05458,POINT (-73.30545 44.71823)
33118,05459,POINT (-73.00456 44.95348)


In [48]:
ny_zctas = all_zctas_centroids.merge(df, how="right", left_on="ZCTA5CE10", right_on="zcta")

In [49]:
ny_zctas

,ZCTA5CE10,geometry,zcta,tmean,date,day,month,year
0,06390,POINT (-71.98836 41.27036),06390,3.021317,01/01/1995,01,01,1995.0
1,10001,POINT (-73.99728 40.75067),10001,0.295794,01/01/1995,01,01,1995.0
2,10002,POINT (-73.98626 40.71576),10002,0.592701,01/01/1995,01,01,1995.0
3,10003,POINT (-73.98918 40.73183),10003,0.295794,01/01/1995,01,01,1995.0
4,10004,POINT (-74.01594 40.69432),10004,0.464349,01/01/1995,01,01,1995.0
...,...,...,...,...,...,...,...,...
654805,14898,POINT (-77.43904 42.05707),14898,-8.191716,31/12/1995,31,12,1995.0
654806,14901,POINT (-76.74429 42.08203),14901,-6.047663,31/12/1995,31,12,1995.0
654807,14903,POINT (-76.87557 42.12526),14903,-6.477991,31/12/1995,31,12,1995.0
654808,14904,POINT (-76.79986 42.06985),14904,-5.841984,31/12/1995,31,12,1995.0


In [50]:
ny_zctas.to_file("ny_tmean_centroids.geojson", drive="GeoJSON")